# GNOD Project

In [1]:
import pickle
import pandas as pd

In [59]:
with open("audio_clustered.txt", "rb") as f:   # Unpickling
    audio_clustered = pickle.load(f)
    
with open("db_songs_clustered.txt", "rb") as f:   # Unpickling
    db_songs = pickle.load(f)
    
with open("top_songs.txt", "rb") as f:   # Unpickling
    top_songs = pickle.load(f)

In [60]:
knn = pickle.load(open('model.p','rb'))
pca = pickle.load(open('pca_model.p','rb'))

In [61]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

secrets_file = open("secrets.txt","r")
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]
        
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [83]:
from random import randint

def gnod (top=top_songs, audio_clustered=audio_clustered, db=db_songs, model=pca):
    song = input("Enter your song: ")
    artist = input("Enter your artist: ")
    # searching the song in our Hot songs list
    for i in range(len(top)):
        # if it's in our list, then recommend another song from the list
        if ((top['title'][i] == song) & (top['artist'][i] == artist)):
            rand_number = randint(0,len(top))
            return(print("I would recommend you to listen to: \"", top['title'][rand_number], "\" by", top['artist'][rand_number]))
    # if is not a Hot song 
    results = sp.search(q=song, limit=50)
    for i in range(len(results["tracks"]["items"])):
        if ((results["tracks"]["items"][i]["artists"][0]["name"]) == (artist)):
            uri = results["tracks"]["items"][i]["uri"]
            audio = pd.DataFrame(sp.audio_features('spotify:track:2JiHR4gCDjXwYVRDmwdk95'))[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
            
            # in the last row I put the song to predict
            X = pd.concat([audio_clustered.drop(['cluster'], axis=1), audio], axis=0)
            X_normalized = pd.DataFrame(StandardScaler().fit_transform(X), columns=audio.columns)
            model.fit(X_normalized)
            X_normalized_pca = model.transform(X_normalized)
            y_pred_pca = pd.Series(knn.predict(X_normalized_pca),name='cluster')

            # this is the predicted value
            pred_cluster = y_pred_pca[len(y_pred_pca)-1]
            songs_recom = db[db['cluster'] == pred_cluster]
            rand_number = randint(0,len(songs_recom))
            recommended_song = songs_recom.loc[:,['title']].iloc[rand_number]
            return(print("Recommended song:", recommended_song))
            
    return("There is no song with this name")

gnod()

Enter your song: Tides
Enter your artist: Ed Sheeran
Recommended song: title    Spiritual Thing
Name: 861, dtype: object
